# Языковые модели на n-граммах

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/api/nltk.util.html
* https://web.stanford.edu/~jurafsky/slp3/3.pdf
* https://www.youtube.com/watch?v=QGT6XTeA3YQ

## Задачи для совместного разбора

In [1]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams

nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anvbakhmatov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

1\. Выделите из текста n-граммы.

In [3]:
text = """Вода это жидкость которая имеет свойство быть водой.
Вода состоит из молекул, которые выглядят как вода"""


In [4]:
tokenizer = RegexpTokenizer(r"\w+")

sents = sent_tokenize(text)
unigrams = []
bigrams = []

for s in sents:
    words = tokenizer.tokenize(s.lower())
    unigrams.extend(ngrams(words, 1))
    bigrams.extend(ngrams(words, 2))
unigrams[:5]


[('вода',), ('это',), ('жидкость',), ('которая',), ('имеет',)]

In [5]:
bigrams[:5]


[('вода', 'это'),
 ('это', 'жидкость'),
 ('жидкость', 'которая'),
 ('которая', 'имеет'),
 ('имеет', 'свойство')]

2. Рассчитайте вероятности  $P(вода)$, $P(это|вода)$, $P(состоит|вода)$.

$$P(w_i) = \frac{C(w_i)}{N}$$
$$P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

In [21]:
unigram = ("вода",)
c = unigrams.count(unigram)
N = len(unigrams)
c / N


0.1875

In [24]:
bigram = ("вода", "это")
c_num = bigrams.count(bigram)
c_denom = unigrams.count(unigram)
c_num, c_denom, c_num / c_denom


(1, 3, 0.3333333333333333)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]
vectorizer = TfidfVectorizer(smooth_idf=False)
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

term = 'this'
term_id = vectorizer.vocabulary_[term]
X.toarray()[:, term_id]

array([0.3645444 , 0.25543054, 0.22314313, 0.3645444 ])

In [35]:
import numpy as np

doc_id = 0
term = "this"

doc_text = corpus[doc_id].lower()

tf = doc_text.count(term) / len(doc_text.split())
df: int = len([doc for doc in corpus if term in doc.lower()])
idf = np.log(len(corpus) / df) + 1
tf * idf

0.2

In [29]:
len([doc for doc in corpus if term in doc.lower()])

4

In [26]:
bigrams.count(("это", "состоит"))

0

3. Рассчитайте вероятности  $P_L(вода)$, $P_L(это|вода)$, $P_L(состоит|вода)$.

$$P_L(w_i) = \frac{C(w_i)+1}{N+V}$$ 

$$P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$$

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `data/moya-semia/Лучше кошки зверя нет 2.csv`. Получите список предложений из сообщений. Приведите предложения к нижнему регистру и удалите все символы, кроме букв и пробелов. Получите список слов (униграмм) и биграмм.

<p class="task" id="2"></p>

2\. Получите распределение вероятностей для униграм $P(w_i) = \frac{C(w_i)}{N}$, где $N$ - количество униграм, $C(w_i)$ - частота использования токена $w_i$. Получите распределение условных вероятностей для биграмм $P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$ ($C(w_{i-1} w_i)$ - частота использования словосочетания $w_{i-1}w_i$).

<p class="task" id="3"></p>

3\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 слов, начинающийся с токена "мой". При генерации текста выбирайте слово с наибольшей вероятностью соответствующего биграмма. Выведите полученный текст на экран.

P(любимый|мой), P(котик|мой), ...


P(x|любимый)

<p class="task" id="4"></p>

4\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

P(любимый|мой), P(котик|мой), ...
0.7             0.3

<p class="task" id="5"></p>

5\. Получите распределение вероятностей для униграм, воспользовавшись сглаживанием Лапласа: $P_L(w_i) = \frac{C(w_i)+1}{N+V}$, где $V$ - количество уникальных униграмм. Получите распределение условных вероятностей для биграмм $P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$

<p class="task" id="6"></p>

6\.Воспользовавшись полученными после сглаживания вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

<p class="task" id="7"></p>

7\. Рассчитайте перплексию для текста "Котя пришел домой с хромой лапой" для четырех моделей: на 1/2-граммах и с/без использования сглаживания Лапласа. Сведите результат в таблицу. Повторите вычисления для текста "После пар я поеду кормить своего кота", используя доступные модели.

$Perplexity(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$

Для модели на униграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i)}$

Для модели на биграммах $P(w_1w_2...w_N) = \Pi_{i=2}^{N}{P(w_i|w_{i-1})}$

In [ ]:
test1 = "После пар я поеду кормить своего кота".lower()
test2 = "Котя пришел домой с хромой лапой".lower()
